# Obtencion de Resultados Finales de la Investigacion

## Importaciones

In [1]:
# Importacion de preguntas y de respuestas
from data.questions import biologia, finanzas, fisica, ia, leyes, matematicas, medicina, quimica, redes, sistemas_operativos 
from data.biologia import respuestas_biologia
from data.finanzas import respuestas_finanzas
from data.fisica import respuestas_fisica
from data.ia import respuestas_ia
from data.leyes import respuestas_leyes
from data.matematicas import respuestas_matematicas
from data.medicina import respuestas_medicina
from data.quimica import respuestas_quimica
from data.redes import redes_respuestas
from data.sistemas_operativos import respuestas_sistemas_operativos

# Importar del sistema original
from src.researcher.graph import build_graph  
from src.researcher.router import Router
from src.researcher.retrieval import Retrieval
from src.researcher.judge_graph import crear_sistema_refinamiento

# Importacion de Librerias
import ollama
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from langchain_core.messages import HumanMessagec:\Users\sceva\anaconda3\envs\rag-Agent\lib\site-packages\tqdm\auto.py:21

['¿Cuál es la diferencia entre una dirección IP pública y una privada?', '¿Qué es el modelo OSI y cuáles son sus 7 capas?', '¿Cómo funciona el protocolo TCP y en qué se diferencia de UDP?', '¿Qué es una máscara de subred y para qué sirve?', '¿Qué es el protocolo ARP y cómo se utiliza en una red local?', '¿Qué diferencia hay entre una red LAN, MAN y WAN?', '¿Qué es el DNS y cómo resuelve nombres de dominio?', '¿Cómo se detectan y previenen ataques de tipo spoofing en redes?', '¿Cómo se implementa NAT y cuál es su función en una red doméstica?', '¿Cuál es la diferencia entre HTTP y HTTPS?', '¿Qué es la inteligencia artificial?', '¿Cuál es la diferencia entre IA débil y IA fuerte?', '¿Qué es un algoritmo de aprendizaje automático?', '¿Qué es una red neuronal artificial?', '¿Qué es el procesamiento del lenguaje natural (NLP)?', '¿Qué es el aprendizaje por refuerzo?', '¿Qué es la visión por computadora?', '¿Qué es el sesgo en los algoritmos de IA y por qué es importante?', '¿Qué es la expli

c:\Users\sceva\anaconda3\envs\rag-Agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuraciones

In [2]:
model = SentenceTransformer('all-mpnet-base-v2')
models_names = ['mistral', 'llama', 'gemma']
temas = ['biologia', 'finanzas', 'fisica', 'ia', 'leyes', 'matematicas', 'medicina', 'quimica', 'redes', 'sistemas_operativos']
listas_preguntas = {
    "biologia": list(zip(biologia, respuestas_biologia)),
    "finanzas": list(zip(finanzas, respuestas_finanzas)),
    "fisica": list(zip(fisica, respuestas_fisica)),
    "ia": list(zip(ia, respuestas_ia)),
    "leyes": list(zip(leyes, respuestas_leyes)),
    "matematicas": list(zip(matematicas, respuestas_matematicas)),
    "medicina": list(zip(medicina, respuestas_medicina)),
    "quimica": list(zip(quimica, respuestas_quimica)),
    "redes": list(zip(redes, redes_respuestas)),
    "sistemas_operativos": list(zip(sistemas_operativos, respuestas_sistemas_operativos))
}

model_mapping = {
    'mistral': 'mistral:7b',
    'llama': 'llama3.1:8b', 
    'gemma': 'gemma3:4b'
}

models = {
    'mistral': lambda prompt: ollama.generate(model='mistral:7b', prompt=prompt)['response'],
    'llama': lambda prompt: ollama.generate(model='llama3.1:8b', prompt=prompt)['response'], 
    'gemma': lambda prompt: ollama.generate(model='gemma3:4b', prompt=prompt)['response']
    
}

print(f"Configuración cargada:")
print(f"   - Modelos: {models_names}")
print(f"   - Temas: {len(temas)}")
print(f"   - Total preguntas: {sum(len(listas_preguntas[tema]) for tema in temas)}")

2025-09-20 23:29:00 - INFO - Use pytorch device_name: cuda
2025-09-20 23:29:00 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
c:\Users\sceva\anaconda3\envs\rag-Agent\lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5060 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5060 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Configuración cargada:
   - Modelos: ['mistral', 'llama', 'gemma']
   - Temas: 10
   - Total preguntas: 100


## Funcion para el calculo de similitud

In [3]:
def calcular_similitud(respuesta_modelo, respuesta_referencia):
    embeddings = model.encode([respuesta_modelo, respuesta_referencia])
    similitud_matrix = cosine_similarity(embeddings)
    return similitud_matrix[0, 1]

## Funcion para probar los modelos

In [4]:
def test_models():
    """Test rápido para verificar que los modelos funcionan"""
    test_prompt = "¿Qué es el agua?"
    print("Probando modelos...")
    
    for name, model_func in models.items():
        try:
            response = model_func(test_prompt)
            print(f"✅ {name}: {response[:80]}...")
        except Exception as e:
            print(f"❌ {name}: Error - {e}")

# Ejecutar test
test_models()

Probando modelos...


2025-09-20 23:29:10 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


✅ mistral:  El agua es una sustancia química que tiene la fórmula química H2O, compuesta po...


2025-09-20 23:29:16 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


✅ llama: El agua es un líquido que ocupa la mayor parte de la Tierra, y es indispensable ...


2025-09-20 23:29:27 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


✅ gemma: El agua es una sustancia increíblemente importante y compleja, y su definición p...


## Sistema de Graphs

In [5]:
async def get_sistema_completo_response(query, model_name):
    """
    Llama al sistema completo (con grafo) para obtener una respuesta.
    
    Args:
        query (str): La pregunta a procesar
        model_name (str): Nombre del modelo a usar
        
    Returns:
        str: La respuesta del sistema
    """
    try:
        # Construir el grafo
        graph = build_graph()
        mapped_model = model_mapping.get(model_name, model_name)
        judge_graph = crear_sistema_refinamiento(model_name=mapped_model)

        # Estado inicial
        state = {
            "messages": [],
            "investigation": True,
            "current_query": "",
            "research_plan": [],
            "retrieval_queries": [],
            "query_category": "",
            "research_collections": [],
            "current_step": "",
            "needs_research": False,
            "retrieval_results": {},
            "context_for_generation": "",
            "research_completed": False,
            "retrieval_obj": Retrieval(persist_directory="./chroma_db"),
            "router_obj": Router(model_name=mapped_model),
            "judge_obj": judge_graph,
            "response_model": mapped_model
        }

        state["router_obj"].retriever = state["retrieval_obj"]

        # Agregar mensaje y ejecutar
        state["messages"].append(HumanMessage(content=query))
        state["current_query"] = query

        # Ejecutar el grafo
        final_state = await graph.ainvoke(state)
        
        # Extraer la respuesta
        if final_state["messages"] and len(final_state["messages"]) > 1:
            ai_response = final_state["messages"][-1]
            return ai_response.content
        else:
            return "No se obtuvo respuesta del sistema completo"
            
    except Exception as e:
        return f"ERROR en sistema completo: {str(e)}"

## Estructura de DataFrames

In [6]:
todas_columnas = []
for tema in temas:
    for i, (pregunta, _) in enumerate(listas_preguntas[tema], 1):
        todas_columnas.append(f"{tema}_p{i}_sim")
        todas_columnas.append(f"{tema}_p{i}_time")

columnas_respuestas = []
for tema in temas:
    for i, (pregunta, _) in enumerate(listas_preguntas[tema], 1):
        columnas_respuestas.append(f"{tema}_p{i}")

# Crear índices para ambos tipos de modelo
indices_ollama = [f"ollama_{model}" for model in models_names]
indices_sistema = [f"sistema_{model}" for model in models_names]
todos_indices = indices_ollama + indices_sistema

# Crear DataFrames
df_distancias = pd.DataFrame(index=todos_indices, columns=todas_columnas, dtype=float)
df_respuestas = pd.DataFrame(index=todos_indices, columns=columnas_respuestas, dtype=object)

print(f"DataFrames creados:")
print(f"   - Distancias: {len(todos_indices)} modelos x {len(todas_columnas)} columnas")
print(f"   - Respuestas: {len(todos_indices)} modelos x {len(columnas_respuestas)} preguntas")
print(f"   - Total experimentos: {len(todos_indices) * len(columnas_respuestas)}")

DataFrames creados:
   - Distancias: 6 modelos x 200 columnas
   - Respuestas: 6 modelos x 100 preguntas
   - Total experimentos: 600


## Pruebas Comparacion y Obtencion de Resultados

In [7]:
async def run_evaluation():
    """Función principal de evaluación"""
    
    # Iniciar evaluación
    start_time = time.time()
    total_experimentos = len(todos_indices) * sum(len(listas_preguntas[tema]) for tema in temas)
    experimento_actual = 0

    for tema in temas:
        print(f"\nProcesando tema: {tema.upper()}")
             
        for i, (pregunta, respuesta_ref) in enumerate(listas_preguntas[tema], 1):
            col_name = f"{tema}_p{i}"
            col_sim = f"{tema}_p{i}_sim"
            col_time = f"{tema}_p{i}_time"
            print(f"  ❓ P{i}: {pregunta[:60]}...")
                     
            # EVALUAR MODELOS OLLAMA DIRECTOS
            for modelo_name, model_func in models.items():
                experimento_actual += 1
                progreso = (experimento_actual / total_experimentos) * 100
                index_ollama = f"ollama_{modelo_name}"
                         
                try:
                    print(f"Ollama-{modelo_name}... ")
                    tiempo_inicio = time.time()
                    
                    respuesta_modelo = model_func(pregunta)
                    tiempo_respuesta = time.time() - tiempo_inicio
                    
                    # Guardar resultados
                    df_respuestas.loc[index_ollama, col_name] = respuesta_modelo
                    similitud = calcular_similitud(respuesta_modelo, respuesta_ref)
                    df_distancias.loc[index_ollama, col_sim] = similitud
                    df_distancias.loc[index_ollama, col_time] = tiempo_respuesta
                    
                    print(f"Sim: {similitud:.3f}, T: {tiempo_respuesta:.2f}s ({progreso:.1f}%)")
                             
                except Exception as e:
                    print(f"Error: {str(e)[:50]}...")
                    df_respuestas.loc[index_ollama, col_name] = f"ERROR: {e}"
                    df_distancias.loc[index_ollama, col_sim] = None
                    df_distancias.loc[index_ollama, col_time] = None

            # EVALUAR SISTEMA COMPLETO
            for modelo_name in models_names:
                experimento_actual += 1
                progreso = (experimento_actual / total_experimentos) * 100
                index_sistema = f"sistema_{modelo_name}"
                         
                try:
                    print(f"Sistema-{modelo_name}... ")
                    tiempo_inicio = time.time()
                    
                    respuesta_modelo = await get_sistema_completo_response(pregunta, modelo_name)
                    tiempo_respuesta = time.time() - tiempo_inicio
                    
                    # Guardar resultados
                    df_respuestas.loc[index_sistema, col_name] = respuesta_modelo
                    similitud = calcular_similitud(respuesta_modelo, respuesta_ref)
                    df_distancias.loc[index_sistema, col_sim] = similitud
                    df_distancias.loc[index_sistema, col_time] = tiempo_respuesta
                    
                    print(f"Sim: {similitud:.3f}, T: {tiempo_respuesta:.2f}s ({progreso:.1f}%)")
                             
                except Exception as e:
                    print(f"Error: {str(e)[:50]}...")
                    df_respuestas.loc[index_sistema, col_name] = f"ERROR: {e}"
                    df_distancias.loc[index_sistema, col_sim] = None
                    df_distancias.loc[index_sistema, col_time] = None

    elapsed_time = time.time() - start_time
    print(f"\nTiempo total: {elapsed_time/60:.2f} minutos")

    return df_distancias, df_respuestas


## Ejecutar Evaluacion

In [8]:
resultado_distancias, resultado_respuestas = await run_evaluation()

# Actualizar las variables globales
df_distancias = resultado_distancias
df_respuestas = resultado_respuestas

# ===== CELDA 9: GUARDAR RESULTADOS Y ESTADÍSTICAS =====
# Guardar archivos CSV
df_distancias.to_csv('resultados_distancia_dual.csv', encoding='utf-8')
df_respuestas.to_csv('resultados_respuestas_dual.csv', encoding='utf-8')

print("Archivos guardados:")
print("   - resultados_distancia_dual.csv")
print("   - resultados_respuestas_dual.csv")

# Estadísticas finales
print(f"\nResumen de resultados:")
print(f"   - Distancias shape: {df_distancias.shape}")
print(f"   - Respuestas shape: {df_respuestas.shape}")

cols_sim = [col for col in df_distancias.columns if col.endswith('_sim')]
cols_time = [col for col in df_distancias.columns if col.endswith('_time')]

# Análisis por tipo de sistema
ollama_rows = [idx for idx in df_distancias.index if idx.startswith('ollama_')]
sistema_rows = [idx for idx in df_distancias.index if idx.startswith('sistema_')]

ollama_similitudes = df_distancias.loc[ollama_rows, cols_sim].notna().sum().sum()
sistema_similitudes = df_distancias.loc[sistema_rows, cols_sim].notna().sum().sum()

print(f"\nComparación de sistemas:")
print(f"   - Ollama directo: {ollama_similitudes} respuestas válidas")
print(f"   - Sistema completo: {sistema_similitudes} respuestas válidas")

if ollama_similitudes > 0:
    avg_sim_ollama = df_distancias.loc[ollama_rows, cols_sim].mean().mean()
    avg_time_ollama = df_distancias.loc[ollama_rows, cols_time].mean().mean()
    print(f"   - Ollama promedio - Sim: {avg_sim_ollama:.3f}, Tiempo: {avg_time_ollama:.2f}s")

if sistema_similitudes > 0:
    avg_sim_sistema = df_distancias.loc[sistema_rows, cols_sim].mean().mean()
    avg_time_sistema = df_distancias.loc[sistema_rows, cols_time].mean().mean()
    print(f"   - Sistema promedio - Sim: {avg_sim_sistema:.3f}, Tiempo: {avg_time_sistema:.2f}s")

# Mostrar muestra de resultados
print(f"\n🔬 Muestra de primeros resultados:")
print("Primeras 6 columnas de distancias:")
if len(df_distancias.columns) >= 6:
    print(df_distancias.iloc[:, :6])
else:
    print(df_distancias)

print("\n🎉 Evaluación completada!")


Procesando tema: BIOLOGIA
  ❓ P1: ¿Qué es una célula?...
Ollama-mistral... 


2025-09-20 23:29:30 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Error: CUDA error: no kernel image is available for execu...
Ollama-llama... 


2025-09-20 23:29:33 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Error: CUDA error: no kernel image is available for execu...
Ollama-gemma... 


2025-09-20 23:29:41 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
Batches:   0%|          | 0/1 [00:00<?, ?it/s]
2025-09-20 23:29:41 - INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5


Error: CUDA error: no kernel image is available for execu...
Sistema-mistral... 


2025-09-20 23:29:45 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-09-20 23:29:45 - INFO - Loading mistral:7b, temperature: 0.1


CancelledError: 

## Analisis Adicional

In [ ]:
print("\nANÁLISIS COMPARATIVO DETALLADO")
print("="*50)

for tema in temas[:3]:  # Primeros 3 temas como muestra
    print(f"\nTema: {tema.upper()}")
    
    # Obtener columnas de similitud para este tema
    cols_tema = [col for col in cols_sim if col.startswith(f"{tema}_")]
    
    if cols_tema:
        # Promedios por modelo y sistema
        for modelo in models_names:
            ollama_idx = f"ollama_{modelo}"
            sistema_idx = f"sistema_{modelo}"
            
            sim_ollama = df_distancias.loc[ollama_idx, cols_tema].mean()
            sim_sistema = df_distancias.loc[sistema_idx, cols_tema].mean()
            
            time_ollama = df_distancias.loc[ollama_idx, [col.replace('_sim', '_time') for col in cols_tema]].mean()
            time_sistema = df_distancias.loc[sistema_idx, [col.replace('_sim', '_time') for col in cols_tema]].mean()
            
            print(f"  🤖 {modelo}:")
            print(f"     Ollama:  Sim={sim_ollama:.3f}, Tiempo={time_ollama:.2f}s")
            print(f"     Sistema: Sim={sim_sistema:.3f}, Tiempo={time_sistema:.2f}s")
            
            if not pd.isna(sim_ollama) and not pd.isna(sim_sistema):
                diff_sim = sim_sistema - sim_ollama
                diff_time = time_sistema - time_ollama
                print(f"     Diferencia: Sim={diff_sim:+.3f}, Tiempo={diff_time:+.2f}s")